In [121]:
import pandas as pd

# Import the raw census results
raw_census = pd.read_csv('./CanadaCensus2016/98-401-X2016045_English_CSV_data.csv',
                         index_col=0,
                         skiprows=lambda r: r != 0 and 260653 > r or 534786 < r,
                         usecols=['GEO_NAME',
                                  'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
                                  'Dim: Sex (3): Member ID: [1]: Total - Sex',
                                  'Dim: Sex (3): Member ID: [2]: Male',
                                  'Dim: Sex (3): Member ID: [3]: Female'],
                         low_memory=False)
raw_census.columns = ['DIM', 'Total', 'Male', 'Female']
raw_census.sort_index(inplace=True)

In [122]:
# Import a riding lookup to convert the riding names from the census names to the election result names
riding_lookup = pd.read_csv('./ridingcensuslookup.csv',
                            header=None,
                            index_col=0,
                            squeeze=True).to_dict()
raw_census.rename(index=riding_lookup, inplace=True)

In [175]:
census_out = pd.DataFrame()

In [176]:
census_out['2016 Population'] = raw_census.loc[raw_census.DIM == 'Population, 2016', 'Total'].astype('int')

In [177]:
def age_range_series(df, age_range):
    return (df.where(df['DIM'] == age_range).groupby(df.index).Total.max() / census_out['2016 Population'] * 100).round(2)

census_out['Pop. % 0-19'] = age_range_series(raw_census, '0 to 14 years') + age_range_series(raw_census, '15 to 19 years')
census_out['Pop. % 20-39'] = (age_range_series(raw_census, '20 to 24 years')
                              + age_range_series(raw_census, '25 to 29 years')
                              + age_range_series(raw_census, '30 to 34 years')
                              + age_range_series(raw_census, '35 to 39 years'))
census_out['Pop. % 40-59'] = (age_range_series(raw_census, '40 to 44 years')
                              + age_range_series(raw_census, '45 to 49 years')
                              + age_range_series(raw_census, '50 to 54 years')
                              + age_range_series(raw_census, '55 to 59 years'))
census_out['Pop. % 60-79'] = (age_range_series(raw_census, '60 to 64 years')
                              + age_range_series(raw_census, '65 to 69 years') 
                              + age_range_series(raw_census, '70 to 74 years')
                              + age_range_series(raw_census, '75 to 79 years'))
census_out['Pop. % 80-'] = (age_range_series(raw_census, '80 to 84 years')
                              + age_range_series(raw_census, '85 to 89 years')
                              + age_range_series(raw_census, '90 to 94 years')
                              + age_range_series(raw_census, '95 to 99 years')
                              + age_range_series(raw_census, '100 years and over'))

In [178]:
census_out['Pop Density (km^2)'] = raw_census.loc[raw_census.DIM == 'Population density per square kilometre', 'Total']

In [186]:
census_out['% Male'] = (raw_census.loc[raw_census.DIM == 'Total - Age groups and average age of the population - 100% data', 'Male'].astype('int') /
                        raw_census.loc[raw_census.DIM == 'Total - Age groups and average age of the population - 100% data', 'Total'] *
                        100).round(2)

In [206]:
census_out['% Married/Common Law'] = (raw_census.loc[raw_census.DIM == 'Married or living common law', 'Total'] /
                                      raw_census.loc[raw_census.DIM == 'Total - Marital status for the population aged 15 years and over - 100% data', 'Total'] *
                                      100).round(2)
census_out['% Divorced'] = (raw_census.loc[raw_census.DIM == 'Divorced', 'Total'] /
                            raw_census.loc[raw_census.DIM == 'Total - Marital status for the population aged 15 years and over - 100% data', 'Total'] *
                            100).round(2)
census_out['% Lone Parent'] = (raw_census.loc[raw_census.DIM == 'Total - Lone-parent census families in private households - 100% data', 'Total'] /
                               raw_census.loc[raw_census.DIM == 'Total number of census families in private households - 100% data', 'Total'] *
                               100).round(2)

In [211]:
census_out['% Immigrant'] = (raw_census.loc[raw_census.DIM == 'Immigrants', 'Total'] /
                             raw_census.loc[raw_census.DIM == 'Total - Immigrant status and period of immigration for the population in private households - 25% sample data', 'Total'] *
                             100).round(2)

In [215]:
census_out['Household Med. Income'] = raw_census.loc[raw_census.DIM == 'Median total income of households in 2015 ($)', 'Total']
census_out['1p Household Med. Income'] = raw_census.loc[raw_census.DIM == 'Median total income of one-person households in 2015 ($)', 'Total']
census_out['2+p Household Med. Income'] =raw_census.loc[raw_census.DIM == 'Median total income of two-or-more-person households in 2015 ($)', 'Total']

In [219]:
census_out.to_csv('formatted_census_data.csv')